In [1]:
from xcube.core.new import new_cube
from xcube.core.store import find_data_store_extensions
from xcube.core.store import get_data_store_params_schema
from xcube.core.store import new_data_store
from xcube.core.store import new_data_writer

from xcube.core.dsio import open_dataset

In [2]:
data_store = new_data_store("s3", root="deep-esdl-output")

In [3]:
list(data_store.get_data_ids())



['black-sea-1x1024x1024.levels',
 'black-sea-1x1024x1024.zarr',
 'black-sea-256x128x128.zarr',
 'esdc-8d-0.25deg-256x128x128-3.0.0.zarr']

In [4]:
ds = data_store.open_data('esdc-8d-0.25deg-256x128x128-3.0.0.zarr')

In [5]:
ds

,Array,Chunk
Bytes,7.64 GiB,16.00 MiB
Shape,"(1978, 720, 1440)","(256, 128, 128)"
Count,2 Graph Layers,576 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,7.64 GiB,16.00 MiB
Shape,"(1978, 720, 1440)","(256, 128, 128)"
Count,2 Graph Layers,576 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


## Create a copy of cube with different chunking

In [6]:
ds.sm.encoding

{'chunks': (256, 128, 128),
 'preferred_chunks': {'time': 256, 'lat': 128, 'lon': 128},
 'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0),
 'filters': None,
 '_FillValue': nan,
 'dtype': dtype('float32')}

In [7]:
time_chunking = 1
lat_chunking = 720
lon_chunking = 512

In [8]:
for variable in ds.data_vars:
    if variable != "crs":
        if ds[variable].encoding['chunks'] != (time_chunking, lat_chunking, lon_chunking):
            ds[variable].encoding['chunks'] = (time_chunking, lat_chunking, lon_chunking)
        print(variable)
        ds[variable] = ds[variable].chunk({'time': time_chunking, 'lat': lat_chunking, 'lon': lon_chunking})
        print(f'{variable} and {ds[variable].encoding["chunks"]}')
        #print(f'{variable} and {ds[variable].chunks}')

aerosol_optical_thickness_550
aerosol_optical_thickness_550 and (1, 720, 512)
air_temperature_2m
air_temperature_2m and (1, 720, 512)
bare_soil_evaporation
bare_soil_evaporation and (1, 720, 512)
burnt_area
burnt_area and (1, 720, 512)
cot
cot and (1, 720, 512)
cth
cth and (1, 720, 512)
ctt
ctt and (1, 720, 512)
evaporation
evaporation and (1, 720, 512)
evaporation_era5
evaporation_era5 and (1, 720, 512)
evaporative_stress
evaporative_stress and (1, 720, 512)
gross_primary_productivity
gross_primary_productivity and (1, 720, 512)
interception_loss
interception_loss and (1, 720, 512)
kndvi
kndvi and (1, 720, 512)
latent_energy
latent_energy and (1, 720, 512)
max_air_temperature_2m
max_air_temperature_2m and (1, 720, 512)
min_air_temperature_2m
min_air_temperature_2m and (1, 720, 512)
nbar_blue
nbar_blue and (1, 720, 512)
nbar_green
nbar_green and (1, 720, 512)
nbar_nir
nbar_nir and (1, 720, 512)
nbar_red
nbar_red and (1, 720, 512)
nbar_swir1
nbar_swir1 and (1, 720, 512)
nbar_swir2
nbar_

In [9]:
ds

<xarray.Dataset>
Dimensions:                            (time: 1978, lat: 720, lon: 1440)
Coordinates:
  * lat                                (lat) float64 -89.88 -89.62 ... 89.88
  * lon                                (lon) float64 -179.9 -179.6 ... 179.9
  * time                               (time) datetime64[ns] 1979-01-05 ... 2...
Data variables: (12/42)
    aerosol_optical_thickness_550      (time, lat, lon) float32 dask.array<chunksize=(1, 720, 512), meta=np.ndarray>
    air_temperature_2m                 (time, lat, lon) float32 dask.array<chunksize=(1, 720, 512), meta=np.ndarray>
    bare_soil_evaporation              (time, lat, lon) float32 dask.array<chunksize=(1, 720, 512), meta=np.ndarray>
    burnt_area                         (time, lat, lon) float64 dask.array<chunksize=(1, 720, 512), meta=np.ndarray>
    cot                                (time, lat, lon) float32 dask.array<chunksize=(1, 720, 512), meta=np.ndarray>
    cth                                (time, lat, lon) float32 dask.array<chunksize=(1, 720, 512), meta=np.ndarray>
    ...                                 ...
    sif_rtsif                          (time, lat, lon) float32 dask.array<chunksize=(1, 720, 512), meta=np.ndarray>
    sm                                 (time, lat, lon) float32 dask.array<chunksize=(1, 720, 512), meta=np.ndarray>
    snow_sublimation                   (time, lat, lon) float32 dask.array<chunksize=(1, 720, 512), meta=np.ndarray>
    surface_moisture                   (time, lat, lon) float32 dask.array<chunksize=(1, 720, 512), meta=np.ndarray>
    terrestrial_ecosystem_respiration  (time, lat, lon) float32 dask.array<chunksize=(1, 720, 512), meta=np.ndarray>
    transpiration                      (time, lat, lon) float32 dask.array<chunksize=(1, 720, 512), meta=np.ndarray>
Attributes: (12/23)
    Conventions:                CF-1.9
    acknowledgment:             All ESDC data providers are acknowledged insi...
    contributor_name:           ['University of Leipzig', 'Max Planck Institu...
    contributor_url:            ['https://www.uni-leipzig.de/', 'https://www....
    creator_name:               ['University of Leipzig', 'Brockmann Consult ...
    creator_url:                ['https://www.uni-leipzig.de/', 'https://www....
    ...                         ...
    publisher_url:              https://www.earthsystemdatalab.net/
    time_coverage_end:          2021-12-31T00:00:00.000000000
    time_coverage_start:        1979-01-05T00:00:00.000000000
    time_period:                8D
    time_period_reported_day:   5.0
    title:                      Earth System Data Cube (ESDC) v3.0.0

In [10]:
output_id = 'esdc-8d-0.25deg-1x720x512-3.0.0.zarr'

In [ ]:
data_store.write_data(ds, output_id)